In [3]:
from transformers import pipeline
pipe = pipeline("text2text-generation", model="facebook/m2m100_418M")

config.json:   0%|          | 0.00/908 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/298 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

In [4]:

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("facebook/m2m100_418M")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/m2m100_418M").to('cuda')

In [5]:
import pandas as pd
import numpy as np
import csv

In [6]:
!nvidia-smi

Sun Apr 14 12:09:18 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0              26W /  70W |   1967MiB / 15360MiB |     27%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [7]:
!pip install transformers
!pip install datasets transformers[sentencepiece] sacrebleu
!pip install nltk

In [8]:
from datasets import load_dataset
import tensorflow as tf
import pandas as pd
import numpy as np

In [10]:
dataset_path = "cfilt/iitb-english-hindi"
train_dataset = load_dataset(dataset_path, split="train[:1000]")

In [11]:
english_sentences = [item['en'] for item in train_dataset['translation']]
hindi_sentences = [item['hi'] for item in train_dataset['translation']]

In [13]:
from timeit import default_timer as timer
start = timer()
translated = []

for index, (eng, hin) in enumerate(zip(english_sentences, hindi_sentences)):
    encoded_en = tokenizer(hin, return_tensors="pt", padding=True, truncation=True).to('cuda')
    generated_tokens = model.generate(**encoded_en, forced_bos_token_id=tokenizer.get_lang_id("en")).to('cuda')
    translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
    print(f"Index: {index}\nHindi: {hin}\nEnglish (Original): {eng}\nEnglish (Translated): {translation}\n")
    translated.append(translation)

end = timer()

Index: 0
Hindi: अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें
English (Original): Give your application an accessibility workout
English (Translated): Give your application the advantage of accessibility exercises

Index: 1
Hindi: एक्सेर्साइसर पहुंचनीयता अन्वेषक
English (Original): Accerciser Accessibility Explorer
English (Translated): Accelerator accessibility

Index: 2
Hindi: निचले पटल के लिए डिफोल्ट प्लग-इन खाका
English (Original): The default plugin layout for the bottom panel
English (Translated): Default plug-in for the lower patel

Index: 3
Hindi: ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका
English (Original): The default plugin layout for the top panel
English (Translated): Default plug-in for the upper plate

Index: 4
Hindi: उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से निष्क्रिय किया गया है
English (Original): A list of plugins that are disabled by default
English (Translated): List of plug-ins that have been deactivated defaultly

Index: 5
Hindi: अवधि को हाइलाइट रकें
English (Orig

In [14]:
print("Time taken:", end - start)

Time taken: 209.68620567200003


In [15]:
!pip install nltk

In [16]:
import nltk
from nltk.translate.bleu_score import corpus_bleu


In [17]:
references = [[eng] for eng in english_sentences]  # Create reference list of lists
candidates = translated  # Model translations

In [18]:
bleu_score = corpus_bleu(references, candidates)
print(f"BLEU score: {bleu_score:.4f}")

BLEU score: 0.4553
